# DeepSurv-Breast-Prognosis
**Team:** ByteRunners

**Competition:** BioFusion Hackathon 2026

**Problem:** Predicting 5-Year Survival Risk in Breast Cancer Patients.

**Dataset:** METABRIC (Nature 2012), obtained from cBioPortal.

In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Also suppress specific sklearn/torch warnings
import os
os.environ['PYTHONWARNINGS'] = 'ignore'

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sksurv.metrics import concordance_index_censored

In [ ]:
# ---------------------------------------------------------
# CELL 2: DATA LOADING & PREPROCESSING
# ---------------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sksurv.util import Surv

# Load pre-filtered dataset (already contains selected clinical + gene expression features)
df = pd.read_csv('data/choosen_data.csv')

# Data Cleaning & Structuring
# 1. Drop rows with missing targets.
# 2. Convert Status to Boolean (Event = True/Dead, Censored = False/Living).
data = df.dropna(subset=['Overall Survival (Months)', 'Overall Survival Status']).copy()

# Remove the patients deceased by other causes (not breast cancer)
data = data[~data["Patient's Vital Status"].str.contains('OTHER', case=False, na=False)]

data['Event'] = data['Overall Survival Status'].astype(str).apply(lambda x: True if 'DECEASED' in x.upper() else False)
data['Time'] = data['Overall Survival (Months)']

# Preprocessing Features
# Drop non-feature columns: PATIENT_ID, target columns, and derived columns
cols_to_drop = ['PATIENT_ID', 'Overall Survival (Months)', 'Overall Survival Status', "Patient's Vital Status", 'Event', 'Time']
X = data.drop([c for c in cols_to_drop if c in data.columns], axis=1)

# Impute missing numerical values with median
num_cols = X.select_dtypes(include=np.number).columns
if len(num_cols) > 0:
    imp_num = SimpleImputer(strategy='median')
    X[num_cols] = imp_num.fit_transform(X[num_cols])

# One-Hot Encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# Constructing the Target Array
# scikit-survival requires a structured array of (Event, Time) tuples.
y = Surv.from_arrays(event=data['Event'].values, time=data['Time'].values)

print(f"Processed Data: {X.shape[0]} samples, {X.shape[1]} features.")
print(f"Features: {list(X.columns[:10])}... (showing first 10)")
print("Target format example:", y[:5])

In [ ]:
# ---------------------------------------------------------
# CELL 3: EXPLORATORY DATA ANALYSIS (Correlation Matrix)
# ---------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

print("📊 Generating Clinical Correlation Matrix...")

# 1. Define the list of features you want to visualize
# (This matches your raw CSV columns)
viz_features = [
    'Age at Diagnosis', 'Chemotherapy', 'Radiotherapy', 'Hormone Therapy',
    'Tumor Size', 'Tumor Stage', 'Neoplasm Histologic Grade',
    'Lymph Nodes Examined Positive', 'Mutation Count', 'Nottingham Prognostic Index',
    'ER Status', 'HER2 Status', 'PR Status'
]

# 2. Create a temporary view of the data (Before One-Hot Encoding)
# We use 'data' (the cleaned dataframe), NOT 'X' (the processed one)
viz_df = data[viz_features].copy()

# 3. THE MAGIC STEP: Convert Text to Numbers Automatically
# This turns "Positive" -> 1, "Negative" -> 0, etc. without you doing anything manually.
for col in viz_df.select_dtypes(include='object').columns:
    viz_df[col] = viz_df[col].astype('category').cat.codes

# 4. Plot
plt.figure(figsize=(12, 10))
corr = viz_df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix (Clinical Features)")
plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------
# CELL 4: FEATURE ENGINEERING (Polynomial Interactions)
# ---------------------------------------------------------
from sklearn.preprocessing import PolynomialFeatures

# Create Interaction Terms (Degree 2)
# This creates: Age*Size, Age*Stage, etc.
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = poly.fit_transform(X)

# Create DataFrame for new features
poly_cols = poly.get_feature_names_out(X.columns)
X_poly_df = pd.DataFrame(X_poly, columns=poly_cols, index=X.index)

# Merge back with original data (Drop duplicates if needed)
# We keep original features + new interactions
X_final = pd.concat([X, X_poly_df], axis=1)

# Remove duplicate columns if any (e.g. original features might be repeated)
X_final = X_final.loc[:, ~X_final.columns.duplicated()]

print(f"New Feature Count: {X_final.shape[1]} (Added interactions like {poly_cols[-3:]})")

# UPDATE TENSORS
X_tensor = torch.tensor(X_final.values.astype(np.float32), dtype=torch.float32)
# Re-normalize since we added new features with large values (squares)
X_tensor = (X_tensor - X_tensor.mean(dim=0)) / (X_tensor.std(dim=0) + 1e-5)

In [ ]:
# ---------------------------------------------------------
# CELL 5: PREPARE DATA FOR PYTORCH
# ---------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert Data to PyTorch Tensors
# PyTorch requires float32 tensors for the neural network.
X_train_tensor = torch.tensor(X_train.values.astype(np.float32), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values.astype(np.float32), dtype=torch.float32)

# Unpacking the Target
# DeepSurv requires separate Tensors for 'Time' and 'Event', not a structured array.
def get_target_tensor(y_struct):
    events = torch.tensor([x[0] for x in y_struct], dtype=torch.float32).unsqueeze(1)
    times = torch.tensor([x[1] for x in y_struct], dtype=torch.float32).unsqueeze(1)
    return times, events

train_times, train_events = get_target_tensor(y_train)
test_times, test_events = get_target_tensor(y_test)

In [ ]:
# ---------------------------------------------------------
# CELL 6: COX LOSS FUNCTION
# ---------------------------------------------------------
def cox_loss(risk_scores, times, events):
    # Cox Partial Likelihood Loss
    # We want to maximize the probability that the patient who died had a higher risk score 
    # than all other patients who were alive at that time (the "Risk Set").
    
    # 1. Sort by time (descending) to easily compute the Risk Set
    idx = times.sort(dim=0, descending=True)[1].squeeze()
    risk_scores = risk_scores[idx]
    events = events[idx]

    # 2. Compute Log-Sum-Exp of risk scores (The "Risk Set" denominator)
    # cumsum allows us to efficiently calculate the sum of risks for everyone 'at risk'
    exp_scores = torch.exp(risk_scores)
    risk_set_sum = torch.cumsum(exp_scores, dim=0) 
    
    # 3. Calculate Log-Likelihood
    # log(risk / sum(risk_at_risk)) = log(risk) - log(sum(risk_at_risk))
    log_likelihood = risk_scores - torch.log(risk_set_sum + 1e-5) # 1e-5 for stability
    
    # 4. Negate because we want to Minimize Loss (Maximize Likelihood)
    # Only events (deaths) contribute to the loss numerator.
    loss = -torch.mean(log_likelihood * events)
    return loss

In [ ]:
# ---------------------------------------------------------
# CELL 7: DeepSurv MODEL SELECTION (GRID SEARCH)
# ---------------------------------------------------------
from sklearn.model_selection import KFold

# 1. Define Candidate Architectures
configs = [
    {"name": "Shallow", "layers": [32, 16], "dropout": 0.1, "lr": 0.005},  
    # Faster convergence
    {"name": "Deep",    "layers": [128, 64, 32], "dropout": 0.3, "lr": 0.001}, 
    # Captures complexity
    {"name": "Robust",  "layers": [64, 64], "dropout": 0.5, "lr": 0.001}     
    # Prevents overfitting
]

# 2. Dynamic Model Builder
class DynamicDeepSurv(nn.Module):
    def __init__(self, input_dim, layers, dropout):
        super(DynamicDeepSurv, self).__init__()
        layer_list = []
        in_nodes = input_dim
        for out_nodes in layers:
            layer_list.append(nn.Linear(in_nodes, out_nodes))
            layer_list.append(nn.SELU()) # SELU is excellent for medical data
            layer_list.append(nn.BatchNorm1d(out_nodes))
            layer_list.append(nn.Dropout(dropout))
            in_nodes = out_nodes
        layer_list.append(nn.Linear(in_nodes, 1))
        self.network = nn.Sequential(*layer_list)

    def forward(self, x):
        return self.network(x)

# 3. K-Fold Cross-Validation Loop
def run_experiment(X, y_times, y_events, configs, k=5):
    # Ensure inputs are CPU tensors for slicing
    X = X.cpu()
    y_times = y_times.cpu()
    y_events = y_events.cpu()
    
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    results = []

    print(f"Starting {k}-Fold Cross-Validation on {len(configs)} Architectures...")
    
    for config in configs:
        fold_scores = []
        print(f"\nTesting Architecture: {config['name']}")
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            # Slice Tensors using the indices
            X_tr, X_val = X[train_idx], X[val_idx]
            t_tr, t_val = y_times[train_idx], y_times[val_idx]
            e_tr, e_val = y_events[train_idx], y_events[val_idx]
            
            # Initialize & Train
            model = DynamicDeepSurv(X.shape[1], config['layers'], config['dropout'])
            optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=1e-4)
            
            for epoch in range(80): # 80 epochs is enough for selection
                model.train()
                risk = model(X_tr)
                loss = cox_loss(risk, t_tr, e_tr)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            # Evaluate
            model.eval()
            with torch.no_grad():
                val_risk = model(X_val)
                
            try:
                # Calculate C-Index on Validation Fold
                c_index = concordance_index_censored(
                    e_val.squeeze().bool().numpy(), 
                    t_val.squeeze().numpy(), 
                    val_risk.squeeze().numpy()
                )[0]
                fold_scores.append(c_index)
            except:
                pass 
        
        avg_score = np.mean(fold_scores)
        print(f"--> Avg C-Index: {avg_score:.4f}")
        results.append((config['name'], avg_score))
        
    return results


experiment_results = run_experiment(X_train_tensor, train_times, train_events, configs)

In [ ]:
# ---------------------------------------------------------
# CELL 8: TRAIN THE WINNING DeepSurv MODEL
# ---------------------------------------------------------
# We selected "Shallow" based on Grid Search result
# Config: Layers=[32, 16], Dropout=0.1, LR=0.005

# 1. Initialize the Best Model
best_model = DynamicDeepSurv(input_dim=X_train_tensor.shape[1], layers=[32, 16], dropout=0.1)

# 2. Define Optimizer (using the winning Learning Rate)
optimizer = optim.Adam(best_model.parameters(), lr=0.005, weight_decay=1e-4)

print("🏆 Training Final Champion Neural Network (Shallow Architecture)...")

# 3. Train on Full Data
epoch_losses = []
for epoch in range(200): # Training for 200 epochs for maximum convergence
    best_model.train()
    
    # Forward Pass
    risk = best_model(X_train_tensor)
    
    # Loss Calculation
    loss = cox_loss(risk, train_times, train_events)
    
    # Backward Pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    epoch_losses.append(loss.item())

print("✅ 'best_model' is now trained and ready for the Ensemble step.")

# Optional: Plot the training curve
plt.figure(figsize=(6,4))
plt.plot(epoch_losses)
plt.title("DeepSurv Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Cox Loss")
plt.show()

In [ ]:
# ---------------------------------------------------------
# CELL 9: RANDOM SURVIVAL FOREST WITH GRID SEARCH
# ---------------------------------------------------------
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.metrics import concordance_index_censored

# 1. Use the Polynomial Features (X_final from previous steps)
# We need to re-split because X_final has the interaction terms
X_train_poly, X_test_poly, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

print(f"Training RSF on {X_train_poly.shape[1]} features (including interactions)...")

# 2. Define Hyperparameter Grid (The "Model Selection" Step)
param_grid = {
    'n_estimators': [500],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [5, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 5, 10]
}

# 3. Run Grid Search (5-Fold CV)
rsf = RandomSurvivalForest(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=rsf,
    param_grid=param_grid,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    n_jobs=-1,
    verbose=1
)

print("Starting Grid Search for Random Forest (this may take 2-3 mins)...")
grid_search.fit(X_train_poly, y_train)

# 4. Get Best Model
best_rsf = grid_search.best_estimator_
print(f"\n✅ Best Parameters: {grid_search.best_params_}")
print(f"✅ Best CV Score: {grid_search.best_score_:.4f}")

# 5. Final Evaluation on Test Set
rsf_score = best_rsf.score(X_test_poly, y_test)
print(f"\n🎯 Optimized RSF C-Index: {rsf_score:.4f}")

In [ ]:
# ---------------------------------------------------------
# CELL 10: ENSEMBLE MODEL (DeepSurv + RSF)
# ---------------------------------------------------------
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_censored
from scipy.stats import zscore
import torch
import torch.optim as optim
import numpy as np

print("🔧 Building Ensemble Model...")

# 1. Re-Create Train/Test Split using the POLYNOMIAL Data (X_final)
X_train_poly, X_test_poly, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

# 2. Convert to PyTorch Tensors (The "Poly" Tensors)
X_train_tensor_poly = torch.tensor(X_train_poly.values.astype(np.float32), dtype=torch.float32)
X_test_tensor_poly = torch.tensor(X_test_poly.values.astype(np.float32), dtype=torch.float32)

# Normalize (Critical for Neural Networks)
mean = X_train_tensor_poly.mean(dim=0)
std = X_train_tensor_poly.std(dim=0) + 1e-5
X_train_tensor_poly = (X_train_tensor_poly - mean) / std
X_test_tensor_poly = (X_test_tensor_poly - mean) / std

# 3. Re-Initialize Model with Correct Input Dimension
input_dim = X_train_tensor_poly.shape[1]
print(f"Model Input Dimension set to: {input_dim}")

best_model = DynamicDeepSurv(input_dim=input_dim, layers=[32, 16], dropout=0.1)
optimizer = optim.Adam(best_model.parameters(), lr=0.005, weight_decay=1e-4)

# 4. Re-Train on the Polynomial Data
print("Re-Training Final Champion Neural Network...")
for epoch in range(200): 
    best_model.train()
    risk = best_model(X_train_tensor_poly)
    loss = cox_loss(risk, train_times, train_events)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 5. Generate Final Predictions
best_model.eval()
with torch.no_grad():
    risk_deep = best_model(X_test_tensor_poly).numpy().flatten()

# 6. Get RSF Predictions
risk_rf = best_rsf.predict(X_test_poly)

# 7. ENSEMBLE: Normalize and Average
risk_deep_norm = zscore(risk_deep)
risk_rf_norm = zscore(risk_rf)

# Weighted Average (0.6 RF + 0.4 Deep)
ensemble_risk = (0.6 * risk_rf_norm) + (0.4 * risk_deep_norm)

# 8. Calculate Final Scores
c_deep = concordance_index_censored(test_events.squeeze().bool().numpy(), test_times.squeeze().numpy(), risk_deep)[0]
c_rf = concordance_index_censored(test_events.squeeze().bool().numpy(), test_times.squeeze().numpy(), risk_rf)[0]
c_ens = concordance_index_censored(test_events.squeeze().bool().numpy(), test_times.squeeze().numpy(), ensemble_risk)[0]

print("\n" + "="*30)
print("🏆 FINAL RESULTS TABLE")
print("="*30)
print(f"Random Forest: {c_rf:.4f}")
print(f"DeepSurv (Poly): {c_deep:.4f}")
print(f"ENSEMBLE: {c_ens:.4f}")
print("="*30)

In [ ]:
# ---------------------------------------------------------
# CELL 11: PREDICTION VISUALIZATION & MODEL COMPARISON
# ---------------------------------------------------------
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# ============================
# 1. Head View of Final Predictions
# ============================

# Extract Event and Time from structured array y_test
event_status = np.array([x[0] for x in y_test])  # Event (True=Deceased, False=Alive)
survival_months = np.array([x[1] for x in y_test])  # Time in months

# Create a comprehensive results DataFrame
results_df = pd.DataFrame({
    'RSF_Risk': risk_rf,
    'DeepSurv_Risk': risk_deep,
    'Ensemble_Risk': ensemble_risk,
    'RSF_Risk_Norm': risk_rf_norm,
    'DeepSurv_Risk_Norm': risk_deep_norm,
    'Survival_Months': survival_months,
    'Event_Status': event_status,
    'Status_Label': ['Deceased' if s else 'Alive' for s in event_status]
})

# Sort by ensemble risk (highest risk first)
results_df_sorted = results_df.sort_values('Ensemble_Risk', ascending=False).reset_index(drop=True)

print("="*80)
print("📊 TOP 15 HIGHEST RISK PATIENTS (Ensemble Prediction)")
print("="*80)
print(results_df_sorted[['RSF_Risk_Norm', 'DeepSurv_Risk_Norm', 'Ensemble_Risk', 
                          'Survival_Months', 'Status_Label']].head(15).to_string())

print("\n" + "="*80)
print("📊 TOP 15 LOWEST RISK PATIENTS (Ensemble Prediction)")
print("="*80)
print(results_df_sorted[['RSF_Risk_Norm', 'DeepSurv_Risk_Norm', 'Ensemble_Risk', 
                          'Survival_Months', 'Status_Label']].tail(15).to_string())

# ============================
# 2. Model Comparison Visualization
# ============================

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('🔬 Three-Model Prediction Comparison: RSF vs DeepSurv vs Ensemble', 
             fontsize=16, fontweight='bold')

# --- Plot 1: Overlapping Distribution of Risk Scores ---
ax1 = axes[0, 0]
ax1.hist(risk_rf_norm, bins=40, alpha=0.5, label=f'RSF (C={c_rf:.4f})', color='blue', edgecolor='black')
ax1.hist(risk_deep_norm, bins=40, alpha=0.5, label=f'DeepSurv (C={c_deep:.4f})', color='green', edgecolor='black')
ax1.hist(ensemble_risk, bins=40, alpha=0.5, label=f'Ensemble (C={c_ens:.4f})', color='red', edgecolor='black')
ax1.set_xlabel('Normalized Risk Score')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Risk Scores (All Models)')
ax1.legend()

# --- Plot 2: Scatter - RSF vs DeepSurv Predictions ---
ax2 = axes[0, 1]
colors = ['red' if s else 'green' for s in event_status]
ax2.scatter(risk_rf_norm, risk_deep_norm, c=colors, alpha=0.6, edgecolors='black', linewidth=0.3)
ax2.plot([-3, 3], [-3, 3], 'k--', alpha=0.5, label='Perfect Agreement')
ax2.set_xlabel('RSF Risk Score (Normalized)')
ax2.set_ylabel('DeepSurv Risk Score (Normalized)')
ax2.set_title('RSF vs DeepSurv Agreement')
legend_elements = [Patch(facecolor='red', label='Deceased'),
                   Patch(facecolor='green', label='Alive')]
ax2.legend(handles=legend_elements, loc='lower right')

# --- Plot 3: Bar Chart - C-Index Comparison ---
ax3 = axes[0, 2]
models = ['Random\nSurvival Forest', 'DeepSurv\n(Neural Net)', 'Ensemble\n(60% RF + 40% Deep)']
c_indices = [c_rf, c_deep, c_ens]
colors_bar = ['#3498db', '#27ae60', '#e74c3c']
bars = ax3.bar(models, c_indices, color=colors_bar, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('C-Index Score')
ax3.set_title('Model Performance Comparison')
ax3.set_ylim(0.5, max(c_indices) + 0.05)
ax3.axhline(y=0.5, color='gray', linestyle='--', alpha=0.7, label='Random Baseline')
for bar, c_val in zip(bars, c_indices):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{c_val:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=11)
ax3.legend()

# --- Plot 4: Patient Ranking Comparison ---
ax4 = axes[1, 0]
patient_indices = np.arange(len(risk_rf_norm))
sorted_idx_ens = np.argsort(ensemble_risk)
ax4.scatter(patient_indices, risk_rf_norm[sorted_idx_ens], alpha=0.6, s=15, label='RSF', color='blue')
ax4.scatter(patient_indices, risk_deep_norm[sorted_idx_ens], alpha=0.6, s=15, label='DeepSurv', color='green')
ax4.scatter(patient_indices, ensemble_risk[sorted_idx_ens], alpha=0.8, s=15, label='Ensemble', color='red')
ax4.set_xlabel('Patient Rank (by Ensemble Risk)')
ax4.set_ylabel('Normalized Risk Score')
ax4.set_title('Risk Predictions Across All Patients')
ax4.legend()

# --- Plot 5: Box Plot by Event Status ---
ax5 = axes[1, 1]
alive_mask = event_status == False
deceased_mask = event_status == True

box_data = [
    risk_rf_norm[alive_mask], risk_rf_norm[deceased_mask],
    risk_deep_norm[alive_mask], risk_deep_norm[deceased_mask],
    ensemble_risk[alive_mask], ensemble_risk[deceased_mask]
]
box_labels = ['RSF\nAlive', 'RSF\nDeceased', 'Deep\nAlive', 'Deep\nDeceased', 'Ens\nAlive', 'Ens\nDeceased']
bp = ax5.boxplot(box_data, labels=box_labels, patch_artist=True)
colors_box = ['lightgreen', 'lightcoral'] * 3
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
ax5.set_ylabel('Normalized Risk Score')
ax5.set_title('Risk Distribution by Survival Status')
ax5.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# --- Plot 6: Risk Group Analysis ---
ax6 = axes[1, 2]
risk_quantiles = pd.qcut(ensemble_risk, q=4, labels=['Low Risk', 'Medium-Low', 'Medium-High', 'High Risk'])
risk_analysis = pd.DataFrame({
    'Risk_Group': risk_quantiles,
    'RSF_Risk': risk_rf_norm,
    'DeepSurv_Risk': risk_deep_norm,
    'Ensemble_Risk': ensemble_risk,
    'Survival_Months': survival_months,
    'Event': event_status
})

group_stats = risk_analysis.groupby('Risk_Group').agg({
    'RSF_Risk': 'mean',
    'DeepSurv_Risk': 'mean', 
    'Ensemble_Risk': 'mean',
    'Event': 'mean'
}).round(3)

x = np.arange(len(group_stats))
width = 0.25
ax6.bar(x - width, group_stats['RSF_Risk'], width, label='RSF', color='#3498db')
ax6.bar(x, group_stats['DeepSurv_Risk'], width, label='DeepSurv', color='#27ae60')
ax6.bar(x + width, group_stats['Ensemble_Risk'], width, label='Ensemble', color='#e74c3c')
ax6.set_xticks(x)
ax6.set_xticklabels(group_stats.index, rotation=15)
ax6.set_ylabel('Mean Risk Score')
ax6.set_title('Mean Risk by Quartile Group')
ax6.legend()

for i, (idx, row) in enumerate(group_stats.iterrows()):
    ax6.text(i, max(row['RSF_Risk'], row['DeepSurv_Risk'], row['Ensemble_Risk']) + 0.1, 
             f'Mort: {row["Event"]*100:.0f}%', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================
# 3. Summary Statistics Table
# ============================
print("\n" + "="*80)
print("📈 MODEL COMPARISON SUMMARY")
print("="*80)
print(f"{'Model':<25} {'C-Index':<12} {'Mean Risk':<15} {'Std Risk':<15}")
print("-"*67)
print(f"{'Random Survival Forest':<25} {c_rf:<12.4f} {risk_rf_norm.mean():<15.4f} {risk_rf_norm.std():<15.4f}")
print(f"{'DeepSurv Neural Network':<25} {c_deep:<12.4f} {risk_deep_norm.mean():<15.4f} {risk_deep_norm.std():<15.4f}")
print(f"{'Ensemble (60% RF + 40% DL)':<25} {c_ens:<12.4f} {ensemble_risk.mean():<15.4f} {ensemble_risk.std():<15.4f}")
print("="*80)

print("\n✅ Model comparison visualization saved as 'model_comparison.png'")

In [ ]:
# ---------------------------------------------------------
# CELL 12: FEATURE IMPORTANCE (Permutation)
# ---------------------------------------------------------
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

print("⏳ Calculating Feature Importance (this takes ~1-2 minutes)...")

# 1. Calculate Importance on Test Set
result = permutation_importance(
    best_rsf, X_test_poly, y_test, n_repeats=5, random_state=42, n_jobs=-1
)

# 2. Sort and Select Top 15 Features
sorted_idx = result.importances_mean.argsort()[-15:]
top_features = X_test_poly.columns[sorted_idx]
top_scores = result.importances[sorted_idx]

# 3. Enhanced Visualization
plt.figure(figsize=(12, 8))

box = plt.boxplot(
    top_scores.T,
    vert=False,
    labels=top_features,
    patch_artist=True,
    widths=0.7
)

# Apply Professional Coloring
for patch in box['boxes']:
    patch.set_facecolor('#2ecc71')
    patch.set_alpha(0.6)
    patch.set_edgecolor('black')

for median in box['medians']:
    median.set_color('black')
    median.set_linewidth(1.5)

# Add Guidelines
plt.grid(True, axis='x', linestyle='--', alpha=0.7)
plt.axvline(x=0, color='red', linestyle='-', linewidth=1, label="No Impact")

# Labels
plt.title("Top 15 Drivers of Survival Prediction (Feature Importance)", fontsize=14, fontweight='bold')
plt.xlabel("Decrease in Model Accuracy (C-Index) when Feature is Removed", fontsize=12)
plt.ylabel("Clinical Features & Interactions", fontsize=12)
plt.tight_layout()
plt.show()

# 4. Print Summary for Report
print("\n" + "="*40)
print("🏆 TOP 5 PREDICTIVE FEATURES")
print("="*40)
for i in range(1, 6):
    idx = sorted_idx[-i]
    print(f"{i}. {X_test_poly.columns[idx]} (Impact: {result.importances_mean[idx]:.4f})")
print("="*40)

In [ ]:
# ---------------------------------------------------------
# CELL 13: SHAP ANALYSIS (Model Explainability)
# ---------------------------------------------------------
import shap
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("🚀 Initializing SHAP Analysis for the Hybrid Ensemble...")

# 1. Define the Ensemble Wrapper Function
def ensemble_predict_wrapper(X_numpy):
    if isinstance(X_numpy, np.ndarray):
        X_df = pd.DataFrame(X_numpy, columns=X_test_poly.columns)
    else:
        X_df = X_numpy

    # Random Forest Prediction
    risk_rf = best_rsf.predict(X_df)

    # DeepSurv Prediction
    X_tensor = torch.tensor(X_df.values.astype(np.float32), dtype=torch.float32)
    train_mean = X_train_tensor_poly.mean(dim=0)
    train_std = X_train_tensor_poly.std(dim=0) + 1e-5
    X_tensor_norm = (X_tensor - train_mean) / train_std

    best_model.eval()
    with torch.no_grad():
        risk_deep = best_model(X_tensor_norm).numpy().flatten()

    return (0.6 * risk_rf) + (0.4 * risk_deep)

# 2. Prepare Background Data (Summary)
print("⚙️  Summarizing Training Data for Baseline...")
X_train_summary = shap.kmeans(X_train_poly, 20)

# 3. Initialize Kernel Explainer on the ENSEMBLE
explainer = shap.KernelExplainer(ensemble_predict_wrapper, X_train_summary)

# 4. Run on the Full Test Set
print(f"⚡ Running SHAP on all {X_test_poly.shape[0]} test patients...")
print("   (This may take 2-5 mins)")
shap_values = explainer.shap_values(X_test_poly)

# 5. Plot the Results
print("🎨 Generating Plots...")

# Summary Plot (Global Feature Importance)
plt.figure()
plt.title("What drives risk in the Hybrid Ensemble?")
shap.summary_plot(shap_values, X_test_poly, show=False)
plt.tight_layout()
plt.show()

# Top Interaction Plot
top_feature = X_test_poly.columns[0]
shap.dependence_plot(top_feature, shap_values, X_test_poly)